In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras
from keras.utils import to_categorical

2024-05-03 17:16:44.659868: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-03 17:16:44.766402: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-03 17:16:44.766467: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-03 17:16:44.780502: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-03 17:16:44.826515: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# csv 파일을 dataframe으로 변환
df_outfit = pd.read_csv('../data/outfit(male)/outfit(male).csv')
df_weather = pd.read_csv('../data/2022-08-01_to_2024-04-30.csv', encoding='cp949')
# 필요한 columns만 추출
df_outfit = df_outfit[['userId', '상의', '아우터', '하의', '신발', '액세서리', '작성일', '체중', '키', '스타일']].copy()
df_weather = df_weather[['평균기온(°C)', '최저기온(°C)', '최고기온(°C)', '강수 계속시간(hr)', '평균 풍속(m/s)', '평균 상대습도(%)', '일시']].copy()

# '작성일'과 '일시' 열을 datetime 형식으로 변환
df_outfit['작성일'] = pd.to_datetime(df_outfit['작성일'], format='%Y년 %m월 %d일')
df_weather['일시'] = pd.to_datetime(df_weather['일시'])

# 두 dataframe을 날짜를 기준으로 병합
df_merged = pd.merge(df_outfit, df_weather, left_on='작성일', right_on='일시')
df_merged = df_merged.drop(['작성일', '일시'], axis=1)
df_merged

KeyError: "['엑세서리'] not in index"

In [42]:
# 강수 계속시간을 비 옴, 비 오지 않음으로 변경
df_merged_sky = df_merged.copy()
df_merged_sky['강수 여부'] = np.where(df_merged_sky['강수 계속시간(hr)'] > 0, 1, 0)
df_merged_sky.drop(['강수 계속시간(hr)'], axis=1, inplace=True)
df_merged_sky

,userId,상의,아우터,하의,신발,엑세서리,체중,키,스타일,평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 상대습도(%),강수 여부
0,1,셔츠/블라우스,NaN,반바지,운동화,NaN,67,172,"캐주얼, 프렌치시크",19.5,12.8,26.7,2.3,50.1,0
1,1,니트,가디건,면바지,구두/로퍼,비니,67,172,"캐주얼, 프렌치시크",5.4,3.5,9.3,2.7,75.6,1
2,1,긴팔 티,NaN,반바지,스니커즈/캔버스,기타 모자,67,172,"캐주얼, 프렌치시크",17.8,15.3,21.9,3.0,74.0,1
3,1,반팔 티,조끼,데님팬츠,스니커즈/캔버스,NaN,67,172,"캐주얼, 프렌치시크",22.8,19.7,26.3,1.9,66.9,0
4,1,반팔 티 2,NaN,반바지,운동화,NaN,67,172,"캐주얼, 프렌치시크",27.6,24.9,31.1,2.2,80.9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,11,반팔 티,재킷,데님팬츠,운동화,기타 모자,60,170,"미니멀, 스트릿, 캐주얼",10.4,3.8,15.2,3.4,57.0,1
546,11,셔츠/블라우스,가디건,데님팬츠,운동화,기타 모자,60,170,"미니멀, 스트릿, 캐주얼",6.7,3.9,9.8,3.7,60.8,1
547,11,반팔 티,점퍼,면바지,운동화,기타 모자,60,170,"미니멀, 스트릿, 캐주얼",7.4,3.2,12.4,2.6,77.3,1
548,11,"반팔 티, 셔츠/블라우스",NaN,데님팬츠,구두/로퍼,NaN,60,170,"미니멀, 스트릿, 캐주얼",7.6,5.0,12.3,2.8,73.0,1


In [43]:
# '상의', '아우터', '하의', '신발', '엑세서리' 열의 결측값을 '~ 없음'으로 대체
columns = ['상의', '아우터', '하의', '신발', '엑세서리']
df_notnull = df_merged_sky.copy()
for column in columns:
    df_notnull[column] = df_merged[column].fillna(column + ' 없음')
df_notnull['평균 풍속(m/s)'].fillna(df_merged_sky['평균 풍속(m/s)'].mean(), inplace=True)
df_notnull.isna().sum()

/tmp/ipykernel_1041/1458509637.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_notnull['평균 풍속(m/s)'].fillna(df_merged_sky['평균 풍속(m/s)'].mean(), inplace=True)


userId        0
상의            0
아우터           0
하의            0
신발            0
엑세서리          0
체중            0
키             0
스타일           0
평균기온(°C)      0
최저기온(°C)      0
최고기온(°C)      0
평균 풍속(m/s)    0
평균 상대습도(%)    0
강수 여부         0
dtype: int64

In [44]:
# 2가 붙은 단어를 두 번 반복하는 함수
def duplicate_word(text):
    words = text.split(', ')
    for i, word in enumerate(words):
        if '2' in word:
            words[i] = word.replace('2', '') + ', ' + word.replace('2', '')
    return ', '.join(words)

In [45]:
# 2가 붙은 단어를 두 번 반복한 dataframe df_dup 생성
df_dup = df_notnull
for column in columns:
    df_dup[columns] = df_notnull[columns].map(duplicate_word)

In [46]:
'''df_dup.iloc[168]'''

'df_dup.iloc[168]'

In [47]:
# 옷의 조합 컬럼 생성 (상의, 아우터, 하의, 신발, 엑세서리의 각 값들을 하나의 문자열로 조합하여 하나의 컬럼으로 만듦)
df_combination = df_dup.copy()
df_combination['옷 조합'] = df_dup['상의'] + ', ' + df_dup['아우터'] + ', ' + df_dup['하의'] + ', ' + df_dup['신발'] + ', ' + df_dup['엑세서리']
df_combination.drop(columns=['상의', '아우터', '하의', '신발', '엑세서리'], inplace=True)

In [48]:
# 옷의 조합 컬럼의 공백 제거
df_combination['옷 조합'] = df_combination['옷 조합'].str.replace(' ', '')

In [49]:
# 쉼표를 기준으로 텍스트를 나누는 함수
def comma_tokenizer(s):
    return s.split(',')

O_vectorizer = CountVectorizer(tokenizer=comma_tokenizer)

O = O_vectorizer.fit_transform(df_combination['옷 조합'])

/home/ksy/anaconda3/envs/ondoset/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [50]:
# multi-hot encoding된 데이터를 numpy array로 변환
df_encoded_O = pd.DataFrame(O.toarray().tolist(), columns=O_vectorizer.get_feature_names_out())
npa_O = np.array(df_encoded_O)
npa_O

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [51]:
'''# multi-hot encoding된 데이터를 다시 텍스트로 변환
df_encoded_O = df_combination.copy()
df_encoded_O['옷 조합'] = O_vectorizer.inverse_transform(npa_O)
df_string_O = df_encoded_O.copy()
df_string_O['옷 조합'] = df_encoded_O['옷 조합'].apply(lambda x: ', '.join(map(str, x)))
df_string_O['옷 조합']'''

"# multi-hot encoding된 데이터를 다시 텍스트로 변환\ndf_encoded_O = df_combination.copy()\ndf_encoded_O['옷 조합'] = O_vectorizer.inverse_transform(npa_O)\ndf_string_O = df_encoded_O.copy()\ndf_string_O['옷 조합'] = df_encoded_O['옷 조합'].apply(lambda x: ', '.join(map(str, x)))\ndf_string_O['옷 조합']"

In [52]:
'''unique_values = df_string_O['옷 조합'].unique()
unique_values'''

"unique_values = df_string_O['옷 조합'].unique()\nunique_values"

In [53]:
'''# 각 고유값을 범주형으로 매핑하는 딕셔너리 생성
category_mapping = {value: i for i, value in enumerate(unique_values)}
df_class_O = df_string_O['옷 조합'].map(category_mapping).astype('category').copy()'''

"# 각 고유값을 범주형으로 매핑하는 딕셔너리 생성\ncategory_mapping = {value: i for i, value in enumerate(unique_values)}\ndf_class_O = df_string_O['옷 조합'].map(category_mapping).astype('category').copy()"

In [54]:
# 스타일 칼럼의 공백 제거
df_combination['스타일'] = df_combination['스타일'].str.replace(' ', '')

In [55]:
# 쉼표를 기준으로 텍스트를 나누는 함수
def comma_tokenizer(s):
    return s.split(',')

S_vectorizer = CountVectorizer(tokenizer=comma_tokenizer)

S = S_vectorizer.fit_transform(df_combination['스타일'])

# multi-hot encoding된 데이터를 numpy array로 변환
df_encoded_S = pd.DataFrame(S.toarray().tolist(), columns=S_vectorizer.get_feature_names_out())
df_encoded_S
df_encoded = pd.concat([df_combination.drop(columns=['옷 조합', '스타일']), df_encoded_S, df_encoded_O], axis=1)
df_encoded

/home/ksy/anaconda3/envs/ondoset/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,userId,체중,키,평균기온(°C),최저기온(°C),최고기온(°C),평균 풍속(m/s),평균 상대습도(%),강수 여부,댄디,...,조끼,집업,카고바지,코트,털모자,트레이닝/조거팬츠,패딩,패딩조끼,하의없음,후드티
0,1,67,172,19.5,12.8,26.7,2.3,50.1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,67,172,5.4,3.5,9.3,2.7,75.6,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,67,172,17.8,15.3,21.9,3.0,74.0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,67,172,22.8,19.7,26.3,1.9,66.9,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1,67,172,27.6,24.9,31.1,2.2,80.9,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,11,60,170,10.4,3.8,15.2,3.4,57.0,1,0,...,0,0,0,0,0,0,0,0,0,0
546,11,60,170,6.7,3.9,9.8,3.7,60.8,1,0,...,0,0,0,0,0,0,0,0,0,0
547,11,60,170,7.4,3.2,12.4,2.6,77.3,1,0,...,0,0,0,0,0,0,0,0,0,0
548,11,60,170,7.6,5.0,12.3,2.8,73.0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
labels_columns = ['평균기온(°C)', '최저기온(°C)', '최고기온(°C)']
inputs_coulumns = ['체중', '키', '평균 풍속(m/s)', '평균 상대습도(%)']

label_scaler = StandardScaler()
input_scaler = StandardScaler()

df_stand_i = df_encoded[['평균기온(°C)', '최저기온(°C)', '최고기온(°C)']].copy()
df_stand_l = df_encoded[['체중', '키', '평균 풍속(m/s)', '평균 상대습도(%)']].copy()

df_stand = df_encoded.copy()

df_stand[labels_columns] = label_scaler.fit_transform(df_encoded[labels_columns])
df_stand[inputs_coulumns] = input_scaler.fit_transform(df_encoded[inputs_coulumns])

In [104]:
df_input = df_stand.drop(['userId','평균기온(°C)', '최저기온(°C)', '최고기온(°C)'], axis=1).copy()
df_input.isna().sum()

체중            0
키             0
평균 풍속(m/s)    0
평균 상대습도(%)    0
강수 여부         0
댄디            0
미니멀           0
밀리터리          0
스트릿           0
아메카지          0
아이비리그         0
캐주얼           0
클래식           0
프레피           0
프렌치시크         0
가디건           0
구두/로퍼         0
기타모자          0
긴팔티           0
니트            0
데님팬츠          0
레더부츠          0
레인부츠          0
마스크           0
맨투맨           0
머플러           0
면바지           0
민소매티          0
바람막이          0
반바지           0
반팔니트          0
반팔셔츠/블라우스     0
반팔티           0
비니            0
샌들/슬리퍼        0
셔츠/블라우스       0
스니커즈/캔버스      0
스카프           0
슬랙스           0
아우터없음         0
양말            0
엑세서리없음        0
운동화           0
장갑            0
장목양말          0
재킷            0
점퍼            0
조끼            0
집업            0
카고바지          0
코트            0
털모자           0
트레이닝/조거팬츠     0
패딩            0
패딩조끼          0
하의없음          0
후드티           0
dtype: int64

In [105]:
df_label = df_stand[['평균기온(°C)', '최저기온(°C)', '최고기온(°C)']].copy()
df_label

,평균기온(°C),최저기온(°C),최고기온(°C)
0,0.475901,0.206380,0.728130
1,-0.892767,-0.672727,-0.963222
2,0.310885,0.442699,0.261550
3,0.796228,0.858620,0.689248
4,1.262158,1.350163,1.155828
...,...,...,...
545,-0.407424,-0.644369,-0.389718
546,-0.766578,-0.634916,-0.914620
547,-0.698630,-0.701085,-0.661889
548,-0.679216,-0.530936,-0.671610


In [106]:
X = np.array(df_input)
Y = np.array(df_label)

In [107]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
print(X_train.shape)
X_train

(440, 57)


array([[-0.92836892, -0.98495233,  0.2071481 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.12818337,  0.97846171,  0.85623196, ...,  0.        ,
         0.        ,  0.        ],
       [-0.25601747, -0.62796796, -0.60420672, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.54416809, -0.80646015, -0.11739383, ...,  0.        ,
         0.        ,  0.        ],
       [-0.92836892, -0.98495233,  0.04487713, ...,  0.        ,
         0.        ,  0.        ],
       [ 3.39389046,  0.26449297, -0.27966479, ...,  0.        ,
         0.        ,  0.        ]])

In [108]:
print(Y_train.shape)
Y_train

(440, 3)


array([[ 0.85446924,  0.81135614,  0.77673166],
       [ 1.01948602,  1.10439169,  0.83505413],
       [-1.04807716, -1.11700683, -0.92434054],
       ...,
       [ 1.06802036,  1.11384445,  1.01974197],
       [ 0.74769367,  0.57503715,  0.83505413],
       [ 0.90300358,  1.12329721,  0.61148464]])

In [130]:
input = keras.Input(shape=(X_train.shape[1],))
x = keras.layers.Dense(512, activation='relu')(input)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(512, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dense(256, activation='relu')(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
output = keras.layers.Dense(Y_train.shape[1])(x)

In [131]:
model = keras.Model(input, output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [132]:
history = model.fit(X_train, Y_train, epochs=20, batch_size=32, validation_data=(X_val, Y_val))

Epoch 1/20
14/14 [==============================] - 2s 20ms/step - loss: 0.9149 - mae: 0.8068 - val_loss: 0.8291 - val_mae: 0.7546
Epoch 2/20
14/14 [==============================] - 0s 12ms/step - loss: 0.7014 - mae: 0.6943 - val_loss: 0.6448 - val_mae: 0.6468
Epoch 3/20
14/14 [==============================] - 0s 12ms/step - loss: 0.5531 - mae: 0.6117 - val_loss: 0.5668 - val_mae: 0.5780
Epoch 4/20
14/14 [==============================] - 0s 12ms/step - loss: 0.5268 - mae: 0.5987 - val_loss: 0.5192 - val_mae: 0.5516
Epoch 5/20
14/14 [==============================] - 0s 11ms/step - loss: 0.4818 - mae: 0.5695 - val_loss: 0.4826 - val_mae: 0.5296
Epoch 6/20
14/14 [==============================] - 0s 12ms/step - loss: 0.4508 - mae: 0.5409 - val_loss: 0.4656 - val_mae: 0.5275
Epoch 7/20
14/14 [==============================] - 0s 11ms/step - loss: 0.4065 - mae: 0.5132 - val_loss: 0.4722 - val_mae: 0.5251
Epoch 8/20
14/14 [==============================] - 0s 13ms/step - loss: 0.3910 - m

In [136]:
result = model.predict(X_val[4:5])
print(f'예측 결과: {label_scaler.inverse_transform(result)}')
print(f'실제 데이터: {input_scaler.inverse_transform(X_val[4:5, 1:5])}, {X_val[4:5, 5:]} 실제 결과: {label_scaler.inverse_transform(Y_val[4:5])} ')

1/1 [==============================] - 0s 32ms/step
예측 결과: [[17.829153 13.785281 22.383121]]
실제 데이터: [[ 59.41089761 176.67872618   2.93138497  81.88474442]], [[0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 1.]] 실제 결과: [[6.2 4.4 9.3]] 
